### 分为四个部分
    an LLM to power your agent - the agent is not exactly the LLM, it’s more like the agent is a program that uses an LLM as its engine.
    
    a system prompt: what the LLM engine will be prompted with to generate its output
    
    a toolbox from which the agent pick tools to execute
    
    a parser to extract from the LLM output which tools are to call and with which arguments 
## 可分成两类

    Code agent
    React agents

pip install transformers[agents]

In [1]:
#这里的LLM可以是任何模型，只要它满足
#it follows the messages format for its input (List[Dict[str, str]]) and returns a str
#it stops generating outputs at the sequences passed in the argument stop

In [ ]:
from huggingface_hub import login, InferenceClient

login("<YOUR_HUGGINGFACEHUB_API_TOKEN>")

client = InferenceClient(model="meta-llama/Meta-Llama-3-70B-Instruct")

def llm_engine(messages, stop_sequences=["Task"]) -> str:
    response = client.chat_completion(messages, stop=stop_sequences, max_tokens=1000)
    answer = response.choices[0].message.content
    return answer

In [ ]:
from transformers import CodeAgent, HfEngine
your_custom_prompt=''
llm_engine = HfEngine(model="meta-llama/Meta-Llama-3-70B-Instruct")
agent = CodeAgent(tools=[],#这个可以是空，没有后续工具
                  llm_engine=llm_engine,#这个是语言模型，有默认模型
                  add_base_tools=True,
                 system_prompt="{your_custom_prompt}"#整体任务的描述
                 )

agent.run(
    "Could you translate this sentence from French, say it out loud and return the audio.",
    sentence="Où est la boulangerie la plus proche?",
)

In [ ]:
#tool :实现了__call__ 

In [ ]:
#自定义tool
from transformers import Tool
from huggingface_hub import list_models

#下面5个东西都要有
class HFModelDownloadsTool(Tool):
    name = "model_download_counter"
    description = (
        "This is a tool that returns the most downloaded model of a given task on the Hugging Face Hub. "
        "It returns the name of the checkpoint."
    )

    inputs = {
        "task": {
            "type": "text",
            "description": "the task category (such as text-classification, depth-estimation, etc)",
        }
    }
    output_type = "text"

    def forward(self, task: str):
        model = next(iter(list_models(filter=task, sort="downloads", direction=-1)))
        return model.id

In [ ]:
agent.toolbox.add_tool(model_download_tool)